In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import json

import pandas as pd

from gzreduction.vote_sources.panoptes_exports import export_to_responses
from gzreduction.vote_sources.panoptes_exports import aggregate
from gzreduction.schemas.jwst_schema import jwst_schema

In [86]:

schema = jwst_schema

In [98]:
classification_export_loc = '/home/walml/repos/gz-panoptes-reduction/data/jwst/jwst-ceers-v0-5-streamlined-clumpy-classifications.csv'
aggregated_loc = '/home/walml/repos/gz-panoptes-reduction/data/jwst/jwst-ceers-v0-5_reduced_votes.csv'

In [88]:
# nrows = 10000
nrows = None
clf = pd.read_csv(classification_export_loc, nrows=nrows)
clf

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,488724185,vrooje,2760.0,9deb947b6b806197cafa,24122,JWST CEERS v0.5 streamlined clumpy,20.26,2023-05-09 19:38:29 UTC,NaN,NaN,"{""source"":""api"",""session"":""a662b8e8b78c1d5d619...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88112645"":{""retired"":null,""RA"":""214.902789"",...",88112645
1,488890955,LucyitSwD,4304.0,8af67c4c7738b7f6bc19,24122,JWST CEERS v0.5 streamlined clumpy,20.26,2023-05-10 14:42:42 UTC,NaN,NaN,"{""source"":""api"",""session"":""23bcfaf9b5308c53bc7...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88115594"":{""retired"":null,""RA"":""214.836407"",...",88115594
2,488891628,sandorkruk,1530.0,25d5da60edeb58b32b60,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-05-10 14:45:14 UTC,NaN,NaN,"{""source"":""api"",""session"":""f13869342311b7c38c6...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88111551"":{""retired"":null,""RA"":""214.905918"",...",88111551
3,488891784,sandorkruk,1530.0,25d5da60edeb58b32b60,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-05-10 14:45:44 UTC,NaN,NaN,"{""source"":""api"",""session"":""f13869342311b7c38c6...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88106122"":{""retired"":null,""RA"":""214.985558"",...",88106122
4,488892110,sandorkruk,1530.0,25d5da60edeb58b32b60,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-05-10 14:47:06 UTC,NaN,NaN,"{""source"":""api"",""session"":""f13869342311b7c38c6...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88107372"":{""retired"":null,""RA"":""214.959103"",...",88107372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228461,511214980,not-logged-in-2f351d9ec581087a385e,NaN,2f351d9ec581087a385e,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-09-15 21:54:33 UTC,NaN,NaN,"{""source"":""api"",""session"":""2329a8e28307b9506ac...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88109487"":{""retired"":null,""RA"":""214.927118"",...",88109487
228462,511215996,not-logged-in-0a98837cf179ec8a09d6,NaN,0a98837cf179ec8a09d6,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-09-15 22:02:25 UTC,NaN,NaN,"{""source"":""api"",""session"":""ceb8d0c0b0c2e2786cb...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88109809"":{""retired"":null,""RA"":""214.937910"",...",88109809
228463,511216005,not-logged-in-0a98837cf179ec8a09d6,NaN,0a98837cf179ec8a09d6,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-09-15 22:02:33 UTC,NaN,NaN,"{""source"":""api"",""session"":""ceb8d0c0b0c2e2786cb...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88110303"":{""retired"":null,""RA"":""214.939919"",...",88110303
228464,511216024,not-logged-in-e7fe4150cf9ac59f24df,NaN,e7fe4150cf9ac59f24df,24122,JWST CEERS v0.5 streamlined clumpy,21.27,2023-09-15 22:02:44 UTC,NaN,NaN,"{""source"":""api"",""session"":""f1d33256e99ce490819...","[{""task"":""T0"",""task_label"":""Is the **central g...","{""88114810"":{""retired"":null,""RA"":""214.832282"",...",88114810


In [89]:
def get_jwst_subject_id(subject_data):
    loaded = json.loads(subject_data)
    inner_dict = loaded[list(loaded.keys())[0]]
    try:
        return inner_dict['id']
    except KeyError:
        return inner_dict['!id']

In [90]:
clf['id_str'] = clf['subject_data'].apply(get_jwst_subject_id)

In [91]:
workflows = [24122]
clf = clf[clf['workflow_id'].isin(workflows)]

In [92]:
df = export_to_responses.explode_annotations(clf, exclude_tasks=['T10'])
df

,task,task_label,value,id_str,user_id,classification_id,created_at,subject_ids,workflow_version
0,T0,Is the **central galaxy** simply smooth and ro...,![star.png](https://panoptes-uploads.zoonivers...,14717,2760.0,488724185,2023-05-09 19:38:29 UTC,88112645,20.26
0,T19,What problem do you see with the image?,![artifact_rotated.png](https://panoptes-uploa...,14717,2760.0,488724185,2023-05-09 19:38:29 UTC,88112645,20.26
1,T0,Is the **central galaxy** simply smooth and ro...,![smooth_triple_flat_new.png](https://panoptes...,24116,4304.0,488890955,2023-05-10 14:42:42 UTC,88115594,20.26
1,T1,How rounded is it?,![in_between_shaped.png](https://panoptes-uplo...,24116,4304.0,488890955,2023-05-10 14:42:42 UTC,88115594,20.26
1,T11,Is the galaxy merging or disturbed?,![no.png](https://panoptes-uploads.zooniverse....,24116,4304.0,488890955,2023-05-10 14:42:42 UTC,88115594,20.26
...,...,...,...,...,...,...,...,...,...
228464,T1,How rounded is it?,![in_between_shaped.png](https://panoptes-uplo...,22481,NaN,511216024,2023-09-15 22:02:44 UTC,88114810,21.27
228464,T11,Is the galaxy merging or disturbed?,![no.png](https://panoptes-uploads.zooniverse....,22481,NaN,511216024,2023-09-15 22:02:44 UTC,88114810,21.27
228465,T0,Is the **central galaxy** simply smooth and ro...,![smooth_triple_flat_new.png](https://panoptes...,31601,NaN,511216033,2023-09-15 22:02:50 UTC,88117918,21.27
228465,T1,How rounded is it?,![cigar_shaped.png](https://panoptes-uploads.z...,31601,NaN,511216033,2023-09-15 22:02:50 UTC,88117918,21.27


In [93]:
question = schema.get_question_from_raw_name('T0')
# question.get_answer_names()
question.get_raw_answer_names()

['smooth', 'features or disk', 'star, artifact, or bad zoom']

In [94]:
cleaned = export_to_responses.clean_exploded_annotations(df, schema)

In [95]:
aggregated = aggregate.responses_to_reduced_votes(cleaned.reset_index(drop=True), schema)

In [96]:
aggregated

question_response,id_str,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_star-artifact-zoom,how-rounded_round,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,bulge-size_dominant_fraction,merging_merger_fraction,merging_major-disturbance_fraction,merging_minor-disturbance_fraction,merging_none_fraction,clumps_yes_fraction,clumps_no_fraction,problem_star_fraction,problem_artifact_fraction,problem_bad-zoom_fraction
0,10000,0,30,2,0,0,0,0,30,0,...,0.1,0.000000,0.100000,0.366667,0.533333,0.666667,0.333333,0.000000,0.000000,1.000000
1,10001,24,2,11,2,21,1,0,2,0,...,0.0,0.000000,0.038462,0.192308,0.769231,0.000000,1.000000,0.181818,0.000000,0.818182
2,10011,7,8,13,5,2,0,0,8,0,...,0.0,0.200000,0.133333,0.133333,0.533333,0.625000,0.375000,0.153846,0.153846,0.692308
3,10015,5,0,31,1,3,1,0,0,0,...,0.0,0.000000,0.200000,0.600000,0.200000,0.000000,0.000000,0.000000,0.225806,0.774194
4,10016,2,0,22,0,2,0,0,0,0,...,0.0,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.136364,0.863636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7674,9992,6,11,7,0,3,3,2,9,1,...,0.0,0.352941,0.117647,0.235294,0.294118,0.636364,0.363636,0.000000,0.428571,0.571429
7675,9993,5,13,5,0,5,0,1,12,0,...,0.0,0.277778,0.444444,0.055556,0.222222,0.692308,0.307692,0.000000,0.400000,0.600000
7676,9995,10,7,5,1,9,0,5,2,4,...,0.0,0.058824,0.000000,0.117647,0.823529,0.000000,1.000000,0.200000,0.200000,0.600000
7677,9996,13,6,7,1,10,2,1,5,0,...,0.2,0.105263,0.052632,0.210526,0.631579,0.166667,0.833333,0.000000,0.000000,1.000000


In [97]:
aggregated

question_response,id_str,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_star-artifact-zoom,how-rounded_round,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,bulge-size_dominant_fraction,merging_merger_fraction,merging_major-disturbance_fraction,merging_minor-disturbance_fraction,merging_none_fraction,clumps_yes_fraction,clumps_no_fraction,problem_star_fraction,problem_artifact_fraction,problem_bad-zoom_fraction
0,10000,0,30,2,0,0,0,0,30,0,...,0.1,0.000000,0.100000,0.366667,0.533333,0.666667,0.333333,0.000000,0.000000,1.000000
1,10001,24,2,11,2,21,1,0,2,0,...,0.0,0.000000,0.038462,0.192308,0.769231,0.000000,1.000000,0.181818,0.000000,0.818182
2,10011,7,8,13,5,2,0,0,8,0,...,0.0,0.200000,0.133333,0.133333,0.533333,0.625000,0.375000,0.153846,0.153846,0.692308
3,10015,5,0,31,1,3,1,0,0,0,...,0.0,0.000000,0.200000,0.600000,0.200000,0.000000,0.000000,0.000000,0.225806,0.774194
4,10016,2,0,22,0,2,0,0,0,0,...,0.0,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.136364,0.863636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7674,9992,6,11,7,0,3,3,2,9,1,...,0.0,0.352941,0.117647,0.235294,0.294118,0.636364,0.363636,0.000000,0.428571,0.571429
7675,9993,5,13,5,0,5,0,1,12,0,...,0.0,0.277778,0.444444,0.055556,0.222222,0.692308,0.307692,0.000000,0.400000,0.600000
7676,9995,10,7,5,1,9,0,5,2,4,...,0.0,0.058824,0.000000,0.117647,0.823529,0.000000,1.000000,0.200000,0.200000,0.600000
7677,9996,13,6,7,1,10,2,1,5,0,...,0.2,0.105263,0.052632,0.210526,0.631579,0.166667,0.833333,0.000000,0.000000,1.000000


In [99]:
aggregated.to_csv(aggregated_loc, index=False)